In [1]:
import os
import json
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
from datasets import (
    Dataset,
    load_from_disk,
    concatenate_datasets,
)


In [3]:
data_path = "../data/wikipedia_documents.json"
df = pd.read_json(data_path)
df

,0,1,2,3,4,5,6,7,8,9,...,60603,60604,60605,60606,60607,60608,60609,60610,60611,60612
text,"이 문서는 나라 목록이며, 전 세계 206개 나라의 각 현황과 주권 승인 정보를 개...",이 목록에 실린 국가 기준은 1933년 몬테비데오 협약 1장을 참고로 하였다. 협정...,현 서울특별시 종로구 서린동 (구 일제 강점기 경기도 경성부 서린정) 출신이다. 친...,"아오조라 문고(靑空文庫, あおぞらぶんこ|아오조라 분고)는 ‘일본어판 구텐베르크 프로...",저자 사망 이후 50년이 지나 저작권이 소멸한 메이지 시대부터 쇼와 시대 초기까지의...,아오조라 문고는 자원봉사로 운영되며 열람 역시 무료이다. 서비스 개시 초반에는 보...,"텍스트 파일을 아오조라 문고에 수록할 때, 텍스트 파일이 갖추어야 할 서식을 '아오...",일본어 표기에 많이 쓰이는 올림문자 (후리가나)는 그대로 올려쓰지 않고 '｜'나 '...,켄 톰슨(Ken Thompson) 등과 함께 최초의 유닉스(Unix) 시스템을 개발...,"주기율표(週期律表, 주기률표, periodic table) 또는 주기표(週期表)는 ...",...,"재래식 전력과 비대칭 전력, 즉 핵까지 갖고 있다한 들, 독자적으로 GPS(GNSS...","소련은 1960년 10월, 알제리를 독립 정부로 최초로 인정해준 나라였다. 소련이 ...","미국 안보 매체, 내셔널 인터레스트지는 한국군이 국방비를 늘렸다고, 비스트 모드(짐...","곰 자체는 페름 지역 숲에 사는 동물이 풍부함을 상징하며, 은색인 이유는 금속, 소...","1967년 소련 통치기 페름 시 의회는 새 문장 공모전을 개최하였으며, 1968년까...","오키나와 현립 박물관·미술관( , Okinawa Prefectural Museum ...","1936년 7월, 오키나와현 교육위원회 부설 향토 박물관(沖縄県教育会附設郷土博物館)...",박물관은 개관 10주년이되는 2017년에 관의 애칭 및 마스코트를 일반인에게 공모했...,단결권 및 단체교섭권 협약(Right to Organise and Collectiv...,이 협약은 부당노동행위 제도를 규율하고 있다. 협약 제1조에서 반노동조합 차별행위로...
corpus_source,위키피디아,위키피디아,위키피디아,위키피디아,위키피디아,위키피디아,위키피디아,위키피디아,위키피디아,위키피디아,...,위키피디아,위키피디아,위키피디아,위키피디아,위키피디아,위키피디아,위키피디아,위키피디아,위키피디아,위키피디아
url,TODO,TODO,TODO,TODO,TODO,TODO,TODO,TODO,TODO,TODO,...,None,None,None,None,None,None,None,None,None,None
domain,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
title,나라 목록,나라 목록,백남준,아오조라 문고,아오조라 문고,아오조라 문고,아오조라 문고,아오조라 문고,데니스 리치,주기율표,...,자주국방론,자주국방론,자주국방론,페름의 문장,페름의 문장,오키나와 현립 박물관·미술관,오키나와 현립 박물관·미술관,오키나와 현립 박물관·미술관,단결권 및 단체교섭권 협약,단결권 및 단체교섭권 협약
author,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
html,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
document_id,0,1,2,3,4,5,6,7,8,9,...,60603,60604,60605,60606,60607,60608,60609,60610,60611,60612


In [11]:
df = df.T
df

,text,corpus_source,url,domain,title,author,html,document_id
0,"이 문서는 나라 목록이며, 전 세계 206개 나라의 각 현황과 주권 승인 정보를 개...",위키피디아,TODO,None,나라 목록,None,None,0
1,이 목록에 실린 국가 기준은 1933년 몬테비데오 협약 1장을 참고로 하였다. 협정...,위키피디아,TODO,None,나라 목록,None,None,1
2,현 서울특별시 종로구 서린동 (구 일제 강점기 경기도 경성부 서린정) 출신이다. 친...,위키피디아,TODO,None,백남준,None,None,2
3,"아오조라 문고(靑空文庫, あおぞらぶんこ|아오조라 분고)는 ‘일본어판 구텐베르크 프로...",위키피디아,TODO,None,아오조라 문고,None,None,3
4,저자 사망 이후 50년이 지나 저작권이 소멸한 메이지 시대부터 쇼와 시대 초기까지의...,위키피디아,TODO,None,아오조라 문고,None,None,4
...,...,...,...,...,...,...,...,...
60608,"오키나와 현립 박물관·미술관( , Okinawa Prefectural Museum ...",위키피디아,None,None,오키나와 현립 박물관·미술관,None,None,60608
60609,"1936년 7월, 오키나와현 교육위원회 부설 향토 박물관(沖縄県教育会附設郷土博物館)...",위키피디아,None,None,오키나와 현립 박물관·미술관,None,None,60609
60610,박물관은 개관 10주년이되는 2017년에 관의 애칭 및 마스코트를 일반인에게 공모했...,위키피디아,None,None,오키나와 현립 박물관·미술관,None,None,60610
60611,단결권 및 단체교섭권 협약(Right to Organise and Collectiv...,위키피디아,None,None,단결권 및 단체교섭권 협약,None,None,60611
